In [7]:
import pandas as pd
import numpy as np
from dtaidistance import dtw
import os
from sklearn.cluster import SpectralClustering
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from __future__ import absolute_import
from __future__ import print_function
from GPUDTW import cuda_dtw, cpu_dtw, dtw_1D_jit2
import numpy
import time

In [8]:
os.getcwd()

'/home/maxime/Documents/WORKSPACES/forecasting_models/cluster/features_clustering'

In [9]:
data = pd.read_csv('../../tests/enc_X_train.csv')

In [10]:
data

,nb_emmergencies_CHU Dijon,O3_FR26005,O3_FR26010,PM10_FR26005,PM10_FR26014,PM25_FR26005,NO2_FR26005,NO2_FR26014,inc_grippe,inc_diarrhee,...,confinement1,confinement2,couvrefeux,ramadan,before_HNFC_moving,during_HNFC_moving,after_HNFC_moving,before_COVID,during_COVID,after_COVID
0,1.431246,-0.103467,-0.075270,-0.011981,-0.065712,0.240391,0.598583,0.237403,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
1,1.195802,-0.688671,-0.371901,0.260223,-0.197136,-0.014353,1.903848,1.344561,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
2,1.431246,-1.524676,-1.431300,0.940733,1.117106,1.480146,1.903848,1.221543,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
3,0.934198,-1.817278,-1.939811,3.118367,3.351317,4.197416,2.164900,1.344561,0.794220,0.267929,...,0,0,0,0,0,0,1,1,0,0
4,1.091160,-1.106674,-0.838037,1.893448,1.905651,2.668952,1.251215,0.237403,3.023719,-0.144729,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,0.515631,0.523537,0.348490,1.212938,1.774227,0.630999,0.729109,1.590596,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1
1149,1.221962,0.857939,0.645121,0.668529,0.591409,0.070562,-0.745839,-0.008631,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1
1150,0.986518,0.356335,0.348490,-0.011981,0.065712,0.410220,0.337530,1.467578,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1
1151,-0.033739,0.021933,0.094234,-0.148083,-0.459985,0.240391,-0.419523,0.360421,1.359752,2.082531,...,0,0,0,0,0,0,1,0,0,1


In [11]:
# Calcul de la matrice de distance DTW
def calculate_dtw_distance_matrix(df):
    num_series = df.shape[1]
    dist_matrix = np.zeros((num_series, num_series))
    
    for i in range(num_series):
        for j in range(i + 1, num_series):
            # Calcul de la distance DTW entre chaque paire de séries
            # dist = dtw.distance(df.iloc[:, i], df.iloc[:, j])
            dist = cuda_dtw (numpy.array(df.iloc[:, i]).reshape(-1, 1), numpy.array(df.iloc[:, j]).reshape(-1, 1))
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist  # La matrice est symétrique

    return pd.DataFrame(dist_matrix, columns=df.columns, index=df.columns)

In [12]:
dtw_matrix = calculate_dtw_distance_matrix(data)
dtw_matrix

{'Max_Grid_X': 2147483647, 'Max_Grid_Y': 65535, 'Max_Threads_pre_Block': 1024, 'Max_share_memeory_per_block': 49152, 'COMPUTE_CAPABILITY_MAJOR': 8, 'total_memory': 8321171456}
grids,block_DimX 1010176 1 1024 1024
Splits [0, 2048]
grid_run T0,Gx,Gy,Tbits  2048 2 1 0.015625


ValueError: setting an array element with a sequence.

In [ ]:
def distance_to_similarity_gaussian(distance_matrix, sigma=1.0):
    # Transformation en similarité avec une fonction gaussienne
    similarity_matrix = np.exp(-distance_matrix**2 / (2 * sigma**2))
    np.fill_diagonal(similarity_matrix.values, 1)  # Similitude maximale sur la diagonale
    return similarity_matrix

In [ ]:
similarity_matrix_gaussian = distance_to_similarity_gaussian(dtw_matrix)
similarity_matrix_gaussian

In [ ]:
def clustering_with_spectral(similarity_matrix, n_clusters=3):
    spectral = SpectralClustering(n_clusters=n_clusters, affinity='precomputed')
    labels = spectral.fit_predict(similarity_matrix)
    return labels

In [ ]:
labels_spectral = clustering_with_spectral(similarity_matrix_gaussian)
labels_spectral

In [ ]:
def tsne_visualization(similarity_matrix):
    # Conversion de la matrice de similarité en matrice de distance
    distance_matrix = 1 - similarity_matrix
    tsne = TSNE(metric='precomputed', n_components=2, perplexity=30)
    tsne_result = tsne.fit_transform(distance_matrix)
    return tsne_result

In [ ]:
# Visualisation t-SNE
tsne_result = tsne_visualization(similarity_matrix_gaussian)
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels)  # `labels` peut venir du clustering
plt.title('t-SNE Visualisation based on Similarity Matrix')
plt.show()